<a href="https://colab.research.google.com/github/ArpitaChatterjee/ChatBot/blob/main/ChatBot_GeniSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [1]:
import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')

#Import JSON file

In [20]:
with open('/content/drive/MyDrive/Colab Notebooks/ChatBot/Intent.json') as f:
  intents= json.load(f)

#Preprocessing Data

- Clean the data
- split them into inputs and target tensors
-build  a tokenizer dictionary and turn sentences into sequences.

The target tensors has a bunch of list with a length of unique title list.


In [21]:
def preprocessing(line):
  line= re.sub(r'[^a-zA-z.?!\']', ' ', line)
  line= re.sub(r'[ ]+', ' ', line)
  return line

In [22]:
#get text and intent title from json data
inputs, targets= [], []
classes= []
intent_doc= {}

for intent in intents['intents']:
  if intent['intent'] not in classes:
    classes.append(intent['intent'])
  if intent['intent'] not in intent_doc:
    intent_doc[intent['intent']] = []
        
  for text in intent['text']:
    inputs.append(preprocessing(text))
    targets.append(intent['intent'])
        
  for response in intent['responses']:
    intent_doc[intent['intent']].append(response)

In [23]:
#tokenize
def tokenize_data(input_list):
  tokenizer= tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')

  tokenizer.fit_on_texts(input_list)
  input_seq = tokenizer.texts_to_sequences(input_list)
  input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')

  return tokenizer, input_seq

#preprocess input data
tokenizer , input_tensor = tokenize_data(inputs)

In [24]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)

In [25]:
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))

input shape: (166, 9) and output shape: (166, 25)


#Building the Model

In [26]:
#hyperparameters
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [27]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 512)         75776     
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              656384    
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 25)                3225      
                                                                 
Total params: 768,281
Trainable params: 768,281
Non-trainable params: 0
________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [28]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])

Epoch 1/50
6/6 [==============================] - 4s 74ms/step - loss: 3.2082 - accuracy: 0.0723
Epoch 2/50
6/6 [==============================] - 0s 69ms/step - loss: 2.7387 - accuracy: 0.2169
Epoch 3/50
6/6 [==============================] - 0s 76ms/step - loss: 2.1069 - accuracy: 0.3855
Epoch 4/50
6/6 [==============================] - 0s 68ms/step - loss: 1.3202 - accuracy: 0.5964
Epoch 5/50
6/6 [==============================] - 0s 73ms/step - loss: 0.7678 - accuracy: 0.7410
Epoch 6/50
6/6 [==============================] - 0s 66ms/step - loss: 0.3780 - accuracy: 0.8976
Epoch 7/50
6/6 [==============================] - 0s 71ms/step - loss: 0.2090 - accuracy: 0.9398
Epoch 8/50
6/6 [==============================] - 0s 67ms/step - loss: 0.1451 - accuracy: 0.9759
Epoch 9/50
6/6 [==============================] - 0s 67ms/step - loss: 0.0710 - accuracy: 0.9759
Epoch 10/50
6/6 [==============================] - 0s 68ms/step - loss: 0.0542 - accuracy: 0.9940
Epoch 11/50
6/6 [============

#**Working of GeniSys**

In [29]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))
    
    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)
    
    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]

# chat with bot
print("Note: Enter 'quit' to break the loop.")
while True:
    input_ = input('You:-- ')
    if input_.lower() == 'quit':
        break
    res, typ = response(input_)
    print('GeniSys:--{}'.format(res))
    print()

Note: Enter 'quit' to break the loop.
You:-- hello
GeniSys:--Hi user, please tell me your username

You:-- My name is Arpita
GeniSys:--They call you <HUMAN>, what can I do for you?

You:-- i want admission
GeniSys:--Please refer to this link, which i hope might help : https://www.srmist.edu.in/content/admission-india-portal

You:-- i want to pay my fees
GeniSys:--Kindly refer to this link to solve your query: https://feekart.srmist.edu.in/srmopp/

You:-- i want to book hostel
GeniSys:--Please refer to this link, which i hope might help : https://www.srmist.edu.in/aboutus/welcome-srm-hostels

You:-- ok
GeniSys:--Bye! Come back again soon.

You:-- quit
